# Train network notebook

This notebook is coded to help train a U-net using atrous (dilated) convolutions.
It can be used either in GUI mode with jupyter or in console mode (TO CHECK).

First, we import the modules.
We also import a set of configuration functions that help us when manipulation config files, which are the backbone of the network.

In [1]:
import json
import os
import time
from AxonDeepSeg.config_tools import *

We define the name of the directory we are going to save our model to.

In [2]:
dir_model_save='conv3_rate11-248_last256'

We also define the configuration of the model.

In [3]:
# Dictionnary with the updates
update_dict={
    'dilation_rate': [[1, 1,1], [1, 1,1], [2, 2,2,4, 4, 8, 8]],
    'convolution_per_layer': [3, 3, 7],
    'features_per_convolution': [[[1, 16], [16, 16], [16, 16]],
  [[16, 32], [32, 32], [32, 32]],
  [[32, 64], [64, 64], [64, 64], [64, 128], [128, 128],[128,256],[256,256]]],
    'size_of_convolutions_per_layer': [[ 5, 5,5],
  [3, 3,3],
  [ 3, 3, 3, 3, 3, 3, 3]],
    'depth':3,
    'batch_size':4
}

config=update_config(default_configuration(),update_dict)

We create variables to be used by the network when training as well as the directory to store the model. If the directory already exists we add a timestamp at the end of the directory name.

In [4]:
path_model = os.path.join('../models/', dir_model_save)
if os.path.exists(path_model): #If the path exists already we add a timestamp at the end of dir name.
    dir_model_save += '_' + str(int(time.time()))[-4:]
    path_model = os.path.join('../models/', dir_model_save)
os.makedirs(path_model)

We then make sure that the config file is valid by using a function we imported. __update_config__ updates the first config (default_configuration()) with the second (config_network) so that the parameters in config_network are at least the parameters needed by the network to run.

In [5]:
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config)

Management of the config file: if it already exists, we load it, else, we write it.

In [6]:
path_config_file=os.path.join(path_model,'config_network.json')
if os.path.exists(path_config_file): # if there is already a configfile
    with open(path_config_file, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_config_file, 'w') as f:
        json.dump(config, f, indent=2)
        config_network=config
   
print 'Chosen parameters :\n'
for param_name, param_value in config_network.iteritems():
    print param_name, ' - ', param_value

Chosen parameters :

trainingset_patchsize  -  512
thresholds  -  [0, 0.2, 0.8]
learning_rate_decay_type  -  polynomial
convolution_per_layer  -  [3, 3, 7]
da-gaussian_blur-sigma_max  -  1.5
da-shifting-percentage_max  -  0.1
da-shifting-activate  -  True
batch_norm_decay_decay_activate  -  True
da-flipping-activate  -  True
batch_norm_decay_starting_decay  -  0.7
n_classes  -  3
learning_rate_decay_activate  -  True
trainingset  -  SEM_3c_512
da-random_rotation-activate  -  False
da-rescaling-activate  -  False
batch_norm_decay_decay_period  -  24000
da-shifting-order  -  0
features_per_convolution  -  [[[1, 16], [16, 16], [16, 16]], [[16, 32], [32, 32], [32, 32]], [[32, 64], [64, 64], [64, 64], [64, 128], [128, 128], [128, 256], [256, 256]]]
batch_norm_decay_ending_decay  -  0.9
da-gaussian_blur-order  -  6
size_of_convolutions_per_layer  -  [[5, 5, 5], [3, 3, 3], [3, 3, 3, 3, 3, 3, 3]]
da-gaussian_blur-activate  -  True
dataset_mean  -  120.95
dilation_rate  -  [[1, 1, 1], [1, 1, 1]

We indicate to the network where to find our trainingset

In [7]:
path_trainingset = os.path.join('../data/', config_network["trainingset"], 'training/')
path_model_init=None

Last part, but not the least: actually training the model!

In [8]:
from AxonDeepSeg.train_network import train_model
train_model(path_trainingset, path_model, config_network, path_model_init=path_model_init,gpu='/gpu:0')

('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 16])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [16, 16])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 2, 'Features: ', [16, 16])
('Size:', 5)
('Layer: ', 1, ' Conv: ', 0, 'Features: ', [16, 32])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 1, 'Features: ', [32, 32])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 2, 'Features: ', [32, 32])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 0, 'Features: ', [32, 64])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 1, 'Features: ', [64, 64])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 2, 'Features: ', [64, 64])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 3, 'Features: ', [64, 128])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 4, 'Features: ', [128, 128])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 5, 'Features: ', [128, 256])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 6, 'Features: ', [256, 256])
('Size:', 3)
('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 16])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [16, 16])
('Size:', 5)
('Lay

2018-01-14 11:53:46.504102-epoch:61-loss:0.4005632544385975-acc:0.8396790623664856
2018-01-14 11:54:13.798349-epoch:62-loss:0.39725868044228385-acc:0.8394079537227235
2018-01-14 11:54:41.743981-epoch:63-loss:0.3895729513003908-acc:0.8444862386275982
2018-01-14 11:55:09.582077-epoch:64-loss:0.4145964939018776-acc:0.8326663313240841
2018-01-14 11:55:37.504571-epoch:65-loss:0.39290565663370597-acc:0.8435921545686393
2018-01-14 11:56:05.662489-epoch:66-loss:0.3861750713710127-acc:0.8462794032590143
2018-01-14 11:56:33.872014-epoch:67-loss:0.3823278166096786-acc:0.8463325479934956
2018-01-14 11:57:02.013037-epoch:68-loss:0.3884326712838535-acc:0.8437893267335563
2018-01-14 11:57:29.113100-epoch:69-loss:0.3845868305913333-acc:0.8458140122479405
2018-01-14 11:57:56.786565-epoch:70-loss:0.4204371047431025-acc:0.8326487089025563
Best accuracy model saved in file: ../models/conv3_rate11-248_last256_7896/best_acc_model.ckpt
2018-01-14 11:58:25.361939-epoch:71-loss:0.42153286420065783-acc:0.837743

2018-01-14 12:35:52.459017-epoch:151-loss:0.4209882185376924-acc:0.8447578652151699
2018-01-14 12:36:20.356947-epoch:152-loss:0.4273653503122-acc:0.8409567249232325
2018-01-14 12:36:48.387538-epoch:153-loss:0.4130052975539504-acc:0.8451005302626511
2018-01-14 12:37:16.616749-epoch:154-loss:0.4158256125861201-acc:0.8408988426471579
2018-01-14 12:37:44.728553-epoch:155-loss:0.41924090118243773-acc:0.8403741203505417
2018-01-14 12:38:12.751906-epoch:156-loss:0.4316859389173574-acc:0.8434353532462285
2018-01-14 12:38:40.757030-epoch:157-loss:0.4288821230674612-acc:0.8393402140716026
2018-01-14 12:39:08.776585-epoch:158-loss:0.4239714279257018-acc:0.8416045575306332
2018-01-14 12:39:37.030552-epoch:159-loss:0.427621678031724-acc:0.8384052103963392
2018-01-14 12:40:05.148773-epoch:160-loss:0.4309661480887183-acc:0.8348058462142944
2018-01-14 12:40:33.318765-epoch:161-loss:0.4344134988455937-acc:0.8367464768475499
2018-01-14 12:41:01.374353-epoch:162-loss:0.4257595097196513-acc:0.843172926327